## COS Method Algorithm for the Heston Model

The COS (Fourier Cosine) method prices European options via an efficient Fourier expansion of the discounted payoff, using the characteristic function of the risk-neutral log-price.

#### 1. **Inputs**

- $S_0$: initial stock price  
- $K$: strike price  
- $r$: risk-free interest rate  
- $\tau$: time to maturity  
- $N$: number of terms in the COS expansion  
- $L$: size parameter for the truncation interval  
- $\varphi(u)$: characteristic function of $\log S_\tau$ under risk-neutral measure (Heston model)  
- $CP$: call ($+1$) or put ($-1$) option flag

---

#### 2. **Truncation Interval**

Compute the truncation interval $[a, b]$ via cumulants of $\log S_\tau$:
$$
a = c_1 - L\sqrt{|c_2|}, \qquad b = c_1 + L\sqrt{|c_2|}
$$
where $c_1$, $c_2$ are the first and second cumulant of $\log S_\tau$.

---

#### 3. **Payoff Coefficients**

For strike $K$, define for $k = 0, 1, \ldots, N-1$:
$$
u_k = \frac{k\pi}{b - a}
$$
and $x_0 = \ln\left(\frac{S_0}{K}\right)$.

Payoff coefficients for calls and puts:
- **Call:**
  $$
  c = 0,\quad d = b, \quad H_k = \frac{2}{b - a} \left( \chi_k - \psi_k \right)
  $$
- **Put:**
  $$
  c = a,\quad d = 0, \quad H_k = \frac{2}{b - a} \left( -\chi_k + \psi_k \right)
  $$

With:
$$
\chi_k = \frac{\cos\left(\frac{k\pi(d-a)}{b-a}\right)e^d - \cos\left(\frac{k\pi(c-a)}{b-a}\right)e^c}{1 + \left(\frac{k\pi}{b-a}\right)^2} 
+ \frac{\frac{k\pi}{b-a} \left[ \sin\left(\frac{k\pi(d-a)}{b-a}\right) - \sin\left(\frac{k\pi(c-a)}{b-a}\right)e^c \right]}{1 + \left(\frac{k\pi}{b-a}\right)^2}
$$

$$
\psi_k = \frac{\sin\left(\frac{k\pi(d-a)}{b-a}\right) - \sin\left(\frac{k\pi(c-a)}{b-a}\right)}{\frac{k\pi}{b-a}}
$$

$$
\psi_0 = d - c
$$

---

#### 4. **COS Pricing Formula**

The option price is given by:
$$
V(0) = e^{-r\tau} \, K \cdot \mathrm{Re}\left[ \sum_{k=0}^{N-1} H_k \, \varphi(u_k) \, e^{i (x_0 - a) u_k} \right]
$$
where the $k=0$ term is halved.

---


In [1]:
import time
import numpy as np

# Chi and Psi helper functions for payoff coefficients
def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a) / (b - a))
    if len(k) > 1:  # avoid division by zero for k=0
        psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c

    chi = (np.cos(k * np.pi * (d - a) / (b - a)) * np.exp(d) 
           - np.cos(k * np.pi * (c - a) / (b - a)) * np.exp(c)
           + (k * np.pi / (b - a)) * (np.sin(k * np.pi * (d - a) / (b - a)) 
           - np.sin(k * np.pi * (c - a) / (b - a)) * np.exp(c))) \
          / (1.0 + (k * np.pi / (b - a)) ** 2)
    return chi, psi

def CallPutCoefficients(CP,a,b,k):
    # if CP == OptionType.CALL:
    if CP.upper() == "CALL":
        c, d = 0.0, b
        chi_k, psi_k = Chi_Psi(a,b,c,d,k)
        if a < b and b < 0.0:
            H_k = np.zeros_like(k)
        else:
            H_k = 2.0/(b-a) * (chi_k - psi_k)
    # elif CP == OptionType.PUT:
    elif CP.upper() == "PUT":
        c, d = a, 0.0
        chi_k, psi_k = Chi_Psi(a,b,c,d,k)
        H_k = 2.0/(b-a) * (-chi_k + psi_k)
    return H_k

# Heston characteristic function
def ChFHestonModel(r, tau, kappa, sigma, vbar, v0, rho):

    # i = 1j
    i = complex(0.0,1.0)

    def cf(u):
        d = np.sqrt((kappa - sigma * rho * i * u)**2 +
                    (u**2 + i * u) * sigma**2)
        g = (kappa - sigma * rho * i * u - d) / (kappa - sigma * rho * i * u + d)

        A = (r * i * u * tau
             + (kappa * vbar * tau / sigma**2) * (kappa - sigma * rho * i * u - d)
             - (2 * kappa * vbar / sigma**2) * np.log((1 - g * np.exp(-d * tau)) / (1 - g)))

        C = ((1 - np.exp(-d * tau)) / (sigma**2 * (1 - g * np.exp(-d * tau)))
             * (kappa - sigma * rho * i * u - d))

        return np.exp(A + C * v0)

    return cf

# COS method for a single strike
def COS_Heston_Price(cf, CP, S0, r, tau, K, N=512, L=12):
    i = complex(0.0,1.0)
    x0 = np.log(S0 / K)
    
    # Truncation range
    a = -L * np.sqrt(tau)
    b =  L * np.sqrt(tau)
    
    k = np.arange(N)
    u = k * np.pi / (b - a)
    H_k = CallPutCoefficients(CP, a, b, k)
    
    mat = np.exp(i * (x0 - a) * u)
    temp = cf(u) * H_k
    temp[0] *= 0.5  # First term halved
    
    value = np.exp(-r * tau) * K * np.real(np.dot(mat, temp))
    return float(value)

# -------------------------
# Example usage
if __name__ == "__main__":
    # Heston params
    m     = 0.5       # moneyness
    S0    = 1.0
    K     = m*S0      # Strike price
    r     = 0.05
    tau   = 1.0
    kappa = 2.0
    sigma = 0.3
    vbar  = 0.05
    v0    = 0.05
    rho   = -0.5
    
    
    start = time.time()
    
    cf = ChFHestonModel(r, tau, kappa, sigma, vbar, v0, rho)

    call_price = COS_Heston_Price(cf, "call", S0, r, tau, K)
    put_price  = COS_Heston_Price(cf, "PUT",  S0, r, tau, K)

    end = time.time()

    print(f"Time taken = {end - start}")

    print("COS Call Price:", np.round(call_price,8))
    print("COS Put Price :", np.round(put_price, 8))

    print("European Call Option Price per unit spot:", np.round(call_price/K, 10))
    print("European Put Option Price per unit spot:", np.round(put_price/K, 10))


Time taken = 0.000621795654296875
COS Call Price: 0.5247089
COS Put Price : 0.00032361
European Call Option Price per unit spot: 1.0494177911
European Put Option Price per unit spot: 0.0006472156
